<a href="https://colab.research.google.com/github/UniVR-DH/LLM-test/blob/main/google_api_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
### Installation on Colab (As Of Oct)
!pip install -q -U langchain transformers bitsandbytes accelerate
!pip install -q -U langchain-community langchain-huggingface langchain-google-genai langgraph


In [7]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

In realms of code, a tale I'll sing,
Of LangChain, a language that's king.
A tapestry of words, it weaves,
A symphony of thought, it conceives.

In structure's embrace, it finds its might,
A hierarchy of blocks, so bright.
From basic elements to complex designs,
LangChain's architecture truly shines.

Its syntax, like a poet's quill,
Flows with grace, devoid of frill.
Indentation guides the reader's eye,
A visual cue that cannot lie.

But LangChain's strength lies not alone
In its structure or syntax, but its tone.
It whispers in the ears of AI,
A language they can comprehend and apply.

With each line of code, a task defined,
LangChain empowers minds, both vast and refined.
From data analysis to image recognition,
Its versatility defies any restriction.

In the hands of masters, it becomes a spell,
Transforming ideas into a digital swell.
Machine learning models take flight,
Guided by LangChain's illuminating light.

But like any language, it has its own quirks,
Its nuances and its su

In [9]:
!pip install kuzu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 12.5 MB/s eta 0:00:00


In [10]:
import kuzu

db = kuzu.Database("test_db")
conn = kuzu.Connection(db)
conn.execute("CREATE NODE TABLE Movie (name STRING, PRIMARY KEY(name))")
conn.execute(
    "CREATE NODE TABLE Person (name STRING, birthDate STRING, PRIMARY KEY(name))"
)
conn.execute("CREATE REL TABLE ActedIn (FROM Person TO Movie)")

In [11]:
conn.execute("CREATE (:Person {name: 'Al Pacino', birthDate: '1940-04-25'})")
conn.execute("CREATE (:Person {name: 'Robert De Niro', birthDate: '1943-08-17'})")
conn.execute("CREATE (:Movie {name: 'The Godfather'})")
conn.execute("CREATE (:Movie {name: 'The Godfather: Part II'})")
conn.execute(
    "CREATE (:Movie {name: 'The Godfather Coda: The Death of Michael Corleone'})"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather Coda: The Death of Michael Corleone' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Robert De Niro' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m)"
)

In [28]:
from langchain.chains import KuzuQAChain
from langchain_community.graphs import KuzuGraph

graph = KuzuGraph(db)
prompt = PromptTemplate.from_template("Remember to put RETURN for the requested variable at the end of the query. Question: {query}")
chain = KuzuQAChain.from_llm(llm, graph=graph, prompt=prompt, verbose=True)

In [26]:
print(graph.get_schema)

Node properties: [{'properties': [('name', 'STRING')], 'label': 'Movie'}, {'properties': [('name', 'STRING'), ('birthDate', 'STRING')], 'label': 'Person'}]
Relationships properties: [{'properties': [], 'label': 'ActedIn'}]
Relationships: ['(:Person)-[:ActedIn]->(:Movie)']



In [29]:
from typing import TYPE_CHECKING, Any, Dict, List, Optional, Sequence, TypeVar, Union
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import BaseCallbackHandler
class CustomHandler(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        formatted_prompts = "\n".join(prompts)
        print(f"Prompt:\n{formatted_prompts}")

handler = CustomHandler()

chain.invoke({"query":"Who played in The Godfather: Part II?"}, {"callbacks":[handler]})



> Entering new KuzuQAChain chain...
Prompt:
Human: Task:Generate Kùzu Cypher statement to query a graph database.

Instructions:
Generate the Kùzu dialect of Cypher with the following rules in mind:
1. Do not omit the relationship pattern. Always use `()-[]->()` instead of `()->()`.
2. Do not include triple backticks ``` in your response. Return only Cypher.
3. Do not return any notes or comments in your response.


Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
Node properties: [{'properties': [('name', 'STRING')], 'label': 'Movie'}, {'properties': [('name', 'STRING'), ('birthDate', 'STRING')], 'label': 'Person'}]
Relationships properties: [{'properties': [], 'label': 'ActedIn'}]
Relationships: ['(:Person)-[:ActedIn]->(:Movie)']

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to c

RuntimeError: Parser exception: mismatched input '(' expecting {ATTACH, USE, CALL, COMMENT_, COPY, EXPORT, IMPORT, DROP, ALTER, EXPLAIN, PROFILE, BEGIN, COMMIT, COMMIT_SKIP_CHECKPOINT, ROLLBACK, ROLLBACK_SKIP_CHECKPOINT, INSTALL, LOAD, OPTIONAL, MATCH, UNWIND, CREATE, MERGE, SET, DETACH, DELETE, WITH, RETURN, SP} (line: 1, offset: 0)
"(:Movie {name: "The Godfather: Part II"})-[:ActedIn]->(:Person)"
 ^